In [ ]:
#| label: afiFig7jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path
import os
import sys
from contextlib import contextmanager
from repo2data.repo2data import Repo2Data

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(__file__).parents[2]
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)

data_dir = dataset_path / "04-B1-02-AFI"
data_file = "fig7.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

B1map_raw = mat_contents["B1map_raw"]
AFIData1 = mat_contents["AFIData1"]
AFIData2 = mat_contents["AFIData2"]
mask = mat_contents["mask"]
xAxis = mat_contents["xAxis"][0]
yAxis = mat_contents["yAxis"][0]

## Plot
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}
# PYTHON CODE
from plotly import tools

# Masking B1 map
B1map_raw = np.asarray(B1map_raw)
mask = np.asarray(mask)
B1map_raw_masked = B1map_raw*mask
B1map_raw_masked[np.isnan(B1map_raw_masked)] = 0


trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=AFIData1,
                   colorscale='Gray',
                   showscale = False,
                   visible=False,
                   name = 'Signal1')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=AFIData2,
                   colorscale='Gray',
                   showscale = False,
                   visible=True,
                   name = 'Signal2')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_raw_masked,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'B1 values')

data=[trace1, trace2, trace3]


updatemenus = list([
    dict(active=1,
         x = 0.09,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = 'Signal 1',
                 method = 'update',
                 args = [{'visible': [True, False, True]},
                         ]),
            dict(label = 'Signal 2',
                 method = 'update',
                 args = [{'visible': [False, True, True]},
                           ]),
        ])
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.07,
            y=1.15,
            showarrow=False,
            text='Input Data',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.60,
            y=1.15,
            showarrow=False,
            text='Raw B<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.12,
            y=1.15,
            showarrow=False,
            text='B<sub>1</sub>',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.58]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.40, 0.98]),
    yaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)